# Lire les données et les afficher (repris du code de Olivier)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split  

path='/Users/marierebiere/Desktop/cours/supaero/4A/Bigdata2/public_dat/'  # si les données sont déjà dans le répertoire courant

input_string_train = path + 'store_train.data'
#df_train = pd.DataFrame([x.split(' ') for x in input_string_train.split('\n')],columns=["ID","Day","Date","Number_of_Customers","Open","Promo","State_Holiday","School_Holiday"],
                  #dtype=int)
                  #{"ID":int,"Day":int,"Date":object,"Number_of_Customers":int,"Open":int,"Promo":int,"State_Holiday":int,"School_Holiday":int})
df_train = pd.read_csv(input_string_train ,sep=" ",header=None, names = ["ID", "Day", "Date", "Number_of_Customers", "Open", "Promo", "State_Holiday", "School_Holiday"])

df_train["Date"]=pd.DataFrame(df_train["Date"],dtype="datetime64[ns]")
df_train["State_Holiday"]=pd.DataFrame(df_train["State_Holiday"],dtype="bool")
df_train["State_Holiday"]=pd.DataFrame(df_train["State_Holiday"],dtype="int")

df_train['Month'] = df_train['Date'].map(lambda d: d.month)
df_train["Month"]=pd.DataFrame(df_train["Month"],dtype=int)
df_train['Day_of_month'] = df_train['Date'].map(lambda d: d.day)
df_train["Day_of_month"]=pd.DataFrame(df_train["Day_of_month"],dtype=int)
df_train['Year'] = df_train['Date'].map(lambda d: d.year)
df_train["Year"]=pd.DataFrame(df_train["Year"],dtype=int)

del df_train["Date"]

# Lire les données de test
input_string_test = path + 'store_test.data'
df_test = pd.read_csv(input_string_test ,sep=" ",header=None, names = ["ID", "Day", "Date", "Number_of_Customers", "Open", "Promo", "State_Holiday", "School_Holiday"])
df_test["Date"]=pd.DataFrame(df_test["Date"],dtype="datetime64[ns]")
df_test["State_Holiday"]=pd.DataFrame(df_test["State_Holiday"],dtype="bool")
df_test["State_Holiday"]=pd.DataFrame(df_test["State_Holiday"],dtype="int")

df_test['Month'] = df_test['Date'].map(lambda d: d.month)
df_test["Month"]=pd.DataFrame(df_test["Month"],dtype=int)
df_test['Day_of_month'] = df_test['Date'].map(lambda d: d.day)
df_test["Day_of_month"]=pd.DataFrame(df_test["Day_of_month"],dtype=int)
df_test['Year'] = df_test['Date'].map(lambda d: d.year)
df_test["Year"]=pd.DataFrame(df_test["Year"],dtype=int)

del df_test["Date"]

##### solutions
input_string_sol = path + 'store_train.solution'
df_sol = pd.read_csv(input_string_sol ,sep=" ",header=None, names = ["Sales"])


df_full_train=pd.concat([df_train,df_sol],axis=1)

print("Données d'entrainement complètes :\n", df_full_train.head(), "\n")
print("Données de test :\n", df_test.head(), "\n")
print("Types des données d'entrainement complètes : \n", df_full_train.dtypes)

Données d'entrainement complètes :
     ID  Day  Number_of_Customers  Open  Promo  State_Holiday  School_Holiday  \
0  625    3                  641     1      1              1               0   
1  293    2                  877     1      1              1               1   
2   39    4                  561     1      1              1               0   
3  676    4                 1584     1      1              1               0   
4  709    3                 1477     1      1              1               0   

   Month  Day_of_month  Year  Sales  
0     11             6  2013   7293  
1      7            16  2013   7060  
2      1            23  2014   4565  
3      9            26  2013   6380  
4      1            22  2014  11647   

Données de test :
     ID  Day  Number_of_Customers  Open  Promo  State_Holiday  School_Holiday  \
0  249    5                  725     1      1              1               0   
1  190    4                  564     1      0              1              

On vérifie que le tableau est bien rempli

In [32]:
print("Nb de jours", df_full_train["Day"].size)
print("Nb de magasins", df_full_train["ID"].size)
print("Nb de Customers", df_full_train["Number_of_Customers"].size)
print("Nb de open", df_full_train["Open"].size)
print("Nb de Promo", df_full_train["Promo"].size)
print("Nb de State_Holiday", df_full_train["State_Holiday"].size)
print("Nb de School_Holiday", df_full_train["School_Holiday"].size)
print("Nb de Sales", df_full_train["Sales"].size)

Day_nul = df_full_train.isna()["Day"]
print(df_full_train.isna())
print(Day_nul[1])
nb_jour_nn_remplis = 0
for i in range(df_full_train["Day"].size-1) :
    if Day_nul[i] :
        nb_jour_nn_remplis +1

print("le nombre de jours non remplis est de ",nb_jour_nn_remplis)


Nb de jours 712045
Nb de magasins 712045
Nb de Customers 712045
Nb de open 712045
Nb de Promo 712045
Nb de State_Holiday 712045
Nb de School_Holiday 712045
Nb de Sales 712045
           ID    Day  Number_of_Customers   Open  Promo  State_Holiday  \
0       False  False                False  False  False          False   
1       False  False                False  False  False          False   
2       False  False                False  False  False          False   
3       False  False                False  False  False          False   
4       False  False                False  False  False          False   
5       False  False                False  False  False          False   
6       False  False                False  False  False          False   
7       False  False                False  False  False          False   
8       False  False                False  False  False          False   
9       False  False                False  False  False          False   
10      Fal

le nombre de jours non remplis est de  0


Découper les données de test et d'apprentissage pour voir l'intéret des différents algorithmes sur notre propre set d'entrainement

## KNN pour savoir si ya des ventes ou pas

Pour celà il faut redéfinir les données de la solution en variable binaire : 0 si pas de vente et 1 si la vente>0.

In [33]:
print(df_sol[:10])
df_sol2=pd.DataFrame(df_sol,dtype="bool")
df_sol2=pd.DataFrame(df_sol2,dtype="int")
print(df_sol2[:10])

from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol2,test_size=0.25)

   Sales
0   7293
1   7060
2   4565
3   6380
4  11647
5   7967
6   7101
7      0
8   7516
9   6295
   Sales
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1


In [34]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
digit_knn=knn.fit(X_train, z_train) 
# Estimation de l'erreur de prévision
# sur l'échantillon test
1-digit_knn.score(X_test,z_test)


/Users/marierebiere/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.0

Le score obtenu est stylé

In [35]:
from sklearn.model_selection import GridSearchCV
# grille de valeurs
param=[{"n_neighbors":list(range(1,15))}]
knn= GridSearchCV(KNeighborsClassifier(),param,cv=5,n_jobs=-1)
digit_knnOpt=knn.fit(X_train, z_train)
# paramètre optimal
digit_knnOpt.best_params_["n_neighbors"]

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


1

In [36]:
z_test_tolist = z_test.values.tolist()

z_pred = digit_knnOpt.predict(X_test)

a = np.array(z_test_tolist) # your x
b = np.array(z_pred) # your y
mse = 0
for i in range(len(a)):
    mse+= (a[i]-b[i])**2/len(a)

mse

array([0.])

yeeeeaaah

## Régression logistique

In [1]:
from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=0.2,random_state=11)

NameError: name 'df_train' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
Sales_logit=logit.fit(X_train, z_train)
z_chap = Sales_logit.predict(X_test)
# Erreur sur l'écahntillon test
#1-titan_logit.score(T_test, z_test)

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(z_test,z_chap))
from sklearn.metrics import r2_score
print("R2=",r2_score(z_test,z_chap))

## Arbre de décision binaire

In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=0.2,random_state=11)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Optimisation de la profondeur de l'arbre
param=[{"max_depth":list(range(2,10))}]
tree= GridSearchCV(DecisionTreeClassifier(),param,cv=10,n_jobs=-1)
tree.fit(X_train, z_train)
z_chap = tree.predict(X_test)

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(z_test,z_chap))
from sklearn.metrics import r2_score
print("R2=",r2_score(z_test,z_chap))

## Random Forest

In [2]:
from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=0.2,random_state=11)

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
# définition des paramètres
#forest = RandomForestClassifier(n_estimators=10, 
#   criterion='gini', max_depth=None,
#   min_samples_split=200, min_samples_leaf=1, 
#   max_features='auto', max_leaf_nodes=None,
#   bootstrap=True, oob_score=True)
forest2 = RandomForestClassifier(n_estimators=200, criterion='entropy')
# apprentissage et erreur out-of-bag
#forest = forest.fit(X_train,z_train)
forest2 = forest2.fit(X_train,z_train)
z_chap = forest2.predict(X_test)
#print(1-forest.oob_score_)

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(z_test,z_chap))
from sklearn.metrics import r2_score
print("R2=",r2_score(z_test,z_chap))

## Pipeline

In [2]:
from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=0.25,random_state=11)

Normalisation des données

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
tf = StandardScaler()
tf.fit(X_train, z_train)
Xt_train = tf.transform(X_train)  
print("Moyenne avant centrage et réduction =", np.mean(X_train))
print("Moyenne après centrage et réduction =", np.mean(Xt_train))

Moyenne avant centrage et réduction = ID                      558.367636
Day                       4.001344
Number_of_Customers     633.226875
Open                      0.830306
Promo                     0.380608
State_Holiday             1.000000
School_Holiday            0.178491
Month                     5.848991
Day_of_month             15.704556
Year                   2013.831731
dtype: float64
Moyenne après centrage et réduction = 5.293830973775642e-15


In [7]:
# Raccourci: Xt = tf.fit_transform(X)
tf.fit_transform(X_train)

array([[-1.35569895e+00,  1.00139560e+00,  1.37410388e-01, ...,
        -1.15648112e+00, -8.01179902e-02,  2.16567649e-01],
       [-6.28712052e-01, -5.01708253e-01, -5.37003039e-01, ...,
        -1.15648112e+00, -4.21260595e-01,  2.16567649e-01],
       [-4.36091592e-01, -6.73634132e-04, -3.19380815e-01, ...,
        -8.56017636e-01, -3.07546394e-01,  2.16567649e-01],
       ...,
       [ 1.29749255e+00,  5.00360985e-01,  1.93431950e-01, ...,
        -1.15648112e+00,  4.88453018e-01,  1.50360286e+00],
       [-7.06381592e-01,  5.00360985e-01,  2.34370785e-01, ...,
        -1.15648112e+00, -3.07546394e-01,  1.50360286e+00],
       [ 6.79243007e-01, -5.01708253e-01, -7.82636044e-01, ...,
        -1.45694460e+00, -1.55840261e+00, -1.07046756e+00]])

In [ ]:
# NB. La standardisation préalable est indispensable pour certains algorithmes
# notamment les SVM
from sklearn.svm import SVC
clf = SVC()
# Calcul des scores (bien classés)
print("Sans standardisation =", clf.fit(X_train, z_train).score(X_test, z_test))
print("Avec standardisation =", clf.fit(tf.transform(X_train), z_train).score(tf.transform(X_test), z_test))

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

In [ ]:
clf = make_pipeline(StandardScaler(), 
                    RFE(RandomForestClassifier(), n_features_to_select=10), 
                    RandomForestClassifier())
clf.fit(X_train, z_train)
z_chap = clf.predict(X_test)
print(clf.predict_proba(X_test)[:5])

In [ ]:
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(z_test,z_chap))
from sklearn.metrics import r2_score
print("R2=",r2_score(z_test,z_chap))

## Composition emboitée

Les données initiales sont unies aux composantes de l'ACP, puis les variables les plus importantes au sens des forêts aléatoires sont sélectionnées avant de servir à l'apprentissage d'un réseau de neurones. Ce n'est sûrement pas une stratégie optimale !

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=200,random_state=11)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE

clf = make_pipeline(
    # Build features
    make_union(
        FunctionTransformer(func=lambda X: X), PCA(),), 
    # Select the best features
    RFE(RandomForestClassifier(), n_features_to_select=10),
    # Train
    MLPClassifier(max_iter=500)
)

clf.fit(X_train, z_train)

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning

In [ ]:
z_chap = clf.predict_proba(X_test)
 
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(z_test,z_chap))
from sklearn.metrics import r2_score
print("R2=",r2_score(z_test,z_chap))

## Régression linéaire sur les composantes principales (repris du code d'Arnaud)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=200,random_state=11)

In [4]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler  
# L'algorithme ds réseaux de neurones nécessite éventuellement une normalisation 
# des variables explicatives avec les commandes ci-dessous
scaler = StandardScaler()  
scaler.fit(X_train)  
Xr_train = scaler.transform(X_train)  
# Meme transformation sur le test
Xr_test = scaler.transform(X_test)

In [9]:
from sklearn import linear_model
# Erreur quadratique moyenne

from sklearn.metrics import mean_squared_error
regLasso = linear_model.Lasso()
regLasso.fit(Xr_train,z_train)
z_chap=regLasso.predict(Xr_test)
print("MSE=",mean_squared_error(z_test,z_chap))
from sklearn.metrics import r2_score
print("R2=",r2_score(z_test,z_chap))
print(prev.size)

MSE= 1784875.9279857238
R2= 0.9029811383154882
200
